<a href="https://colab.research.google.com/github/CarolineAndradeR/Python/blob/master/Regress%C3%A3o_Linear.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Conceito**



**Definição** - O modelo de regressão busca ajustar a melhor reta em uma nuvem de pontos. Quando o objetivo for analisar/prever saídas utilizado variáveis quantitativas (renda, peso) a técnica de regressão deve ser utilizada. A regressão busca minimizar os erros.


**Regressão Simples** - Uma variavel para projetar uma outra variavel.

**Regressão Múltipla** - Divesas variaveis para projetar uma outra.

---
Termos utilizados.

**Machine Learning**------**Estatística**---------------------------**Reconhecimento de Padrões**     
Inputs-------------------------Variáveis Independetes----------------Features

Feature Engineering---Transformação de Variáveis--------Feature engineering

Outputs-----------------------Variáveis Dependentes---------------Outputs

Amostra de Treino------Análise in-the-sample-----------------Amostra de Treino

Amostra de Teste-------Análise out-of-sample-----------------Amostra de Treino

---

**Outlier -** Quando se tem um ponto totalmente fora do padrão.
**RMSE -** É utilizada para validar modelos supervisionados com objetivos de regressão. Quanto maior o RMSE, maior o erro e com isso pior é o modelo.

# **Pratica**



In [1]:
# Modulos gerias
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

# Modulos de análise de regressão
# Treinar o modelo e testar em outro conjunto de dados
from sklearn.model_selection import train_test_split
# Algoritimo de Regressão
from sklearn.linear_model import LinearRegression
import statsmodels.formula.api as sm
# Medida para verificar se o modelo funcionou bem
from sklearn.metrics import mean_squared_error

In [2]:
#Iniciando montagem do arquivo com o drive
from google.colab import drive
drive.mount('/content/drive')

import pandas as pd

dados_gastos_cartao = '/content/drive/MyDrive/Asset_FIAP/base_gastos_cartao.csv'
dados_gastos_cartao = pd.read_csv(dados_gastos_cartao)
dados_gastos_cartao.head()

Mounted at /content/drive


,gastos_cartao,idade,renda,impostos,segmento
0,510,35,1120,60,C
1,490,30,1120,60,C
2,470,32,1040,60,C
3,460,31,1200,60,C
4,500,36,1120,60,C


In [3]:
# Dados treino e teste
# Test_size - valor da divsão para teste
# 34251 - Manter a divisão dos valores
train_data, test_data = train_test_split(dados_gastos_cartao,
                                         test_size = 0.33,
                                         random_state = 34251)
# numero de linhas para cada dataset
dados_gastos_cartao.shape[0], train_data.shape[0], test_data.shape[0]

(150, 100, 50)

In [4]:
# Treino
# Uma única variavel de renda (regressão linear simples)
# Verificar se os gastos de cartão é explicado pela renda da pessoa
X = train_data['renda'].values.reshape(-1, 1)
y = train_data['gastos_cartao']

# Teste
X_test = test_data['renda'].values.reshape(-1, 1)
y_test = test_data['gastos_cartao']

# Treinar o modelo
modelo_1 = LinearRegression().fit(X, y)

In [5]:
# Coeficiente e o valor de beta, um valor medio e o valor do coeficiente 
modelo_1.intercept_, modelo_1.coef_[0]
# r2 verifica se o modelo ficou bem ajustado com os dados
modelo_1.score(X, y)

0.7626015444308568